<a href="https://colab.research.google.com/github/KlaraDewitte/Machine_Learning_course_UGent_D012554_kaggle/blob/master/Copy_of_Poging3SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
trainset = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/Machine_Learning_course_UGent_D012554_kaggle/master/eeg_train.csv")

testset = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/Machine_Learning_course_UGent_D012554_kaggle/master/eeg_test.csv")

In [0]:
labels = trainset.pop('label')
indices = testset.pop('index')

In [0]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
scaler.fit(trainset)
trainset_scaled = pd.DataFrame(scaler.transform(trainset))
testset_scaled = pd.DataFrame(scaler.transform(testset))


In [0]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.model_selection import cross_val_score

In [0]:
X = trainset_scaled
y = labels

In [7]:
OptiC = 0
Possibilities = ['linear', 'poly', 'rbf', 'sigmoid']
search_space = [0.01,0.1,1,10,100, 1000]
params = dict(C=search_space)

for pos in Possibilities:
  print(pos)

  clf=SVC(kernel=pos)
  grid_search = GridSearchCV(clf, param_grid=params)
  grid_search.fit(X, y)

  OptiC=grid_search.best_estimator_.C
  print('Optimale C =' + str(OptiC))

  clf=SVC(OptiC, kernel=pos)
  clf.fit(X,y)

  score_auc = np.mean(cross_val_score(clf,X,y,cv=10,scoring='roc_auc'))

  print('auc=' + str(score_auc))


linear
Optimale C =1
auc=0.67434634987846
poly
Optimale C =1000
auc=0.9174034579080452
rbf
Optimale C =100
auc=0.9593785551858949
sigmoid
Optimale C =0.01
auc=0.5926152053858476


In [8]:
clf=SVC(C=100, kernel='rbf', probability=True)
clf.fit(X,y)
predictions = clf.predict_proba(testset_scaled)[:,1]
print(predictions)

[3.06957080e-05 7.21717299e-01 4.93875338e-03 ... 9.83058489e-01
 5.55908771e-01 1.45349844e-02]


In [9]:
dfpredictions= pd.DataFrame(({'index': indices, 'label':predictions}))
print(dfpredictions)

       index     label
0          0  0.000031
1          1  0.721717
2          2  0.004939
3          3  0.989246
4          4  1.000000
...      ...       ...
12887  12887  0.066238
12888  12888  0.999986
12889  12889  0.983058
12890  12890  0.555909
12891  12891  0.014535

[12892 rows x 2 columns]


In [0]:
filename = "my_prediction_results_Poging3SVM.csv"

#make sure to not write the Pandas index column (index=False)
dfpredictions.to_csv(filename,index=False)